# A simple UKF + CSS simulation

In this notebook:
* A 3U CubeSat with 6 Coarse Sun Sensor (CSS) rotating in with constant angular velocity around local and global coordinate systems
* A Simulator to model the state-variables (position, attitude, velocity, and angular velocity) and sensor-observation (light intensity)

_Here we use Homogeneous Coordinates and Unit Quaternions_

In [ ]:
%matplotlib inline

import numpy as np
import transforms3d as tf
from math import pi as PI
from src.unscented_kalman_filter import UKF

import matplotlib.pyplot as plt
import seaborn as sns 

from ipywidgets import interact, interactive, IntSlider, FloatSlider, fixed

## Helpers

In [ ]:
def h(sensors, light_source):
    '''
    Inputs:
    - sensors: A numpy array of shape [num_sensors, 3]
    - light_source: A numpy array of shape [3]
    '''
    def f(state):
        '''
        Inputs:
        - state: position (3), attitude (4), velocity (3), and angular-velocity (4)        
        '''
        
        # map light_source's position from Homogeneous Coordinates to Cartesian Coordinate
        p = light_source[:-1] / light_source[-1]
        # a unit vector pointing to the light_source in local coordinates
        u = p - state[:3]
        u = u / np.linalg.norm(u)
        # quaternion to rotation matrix
        r = tf.quaternions.quat2mat(state[3:7])
        # normal vector of sensors in local coordinates
        n = np.dot(sensors, r.T)
        # dot product between the normal vector to sensor and unit vector to the light_source
        d = np.dot(n, u)
        # reshold
        d[d < 0] = 0
        
        return d
    
    return f

In [ ]:
class Observation:
    def __init__(self, sensors, light_source, std, h):
        '''
        Inputs:
        - sensors: A numpy array of shape [num_sensors, 3]
        - light_source: A numpy array of shape [3]
        - std: indicates the error in the sensor observation
        - h: a function that maps from state-space to observation-space
        '''
        self.h = h(sensors, light_source)
        self.std = std
        
    def measure(self, state):
        z = self.h(state)
        return z + np.random.randn(*z.shape) * self.std

In [ ]:
class Simulator:
    def __init__(self, x0, w_std):
        '''
        Inputs:
        - x0: initial state, [position (3), attitude (4), velocity (3), and angular-velocity (4)]
        - w_std: indicates the error in the angular velocity
        '''
        self.X = x0
        self.w_std = w_std
        
    def step(self):
        # TODO
        pass

## Parameters

In [ ]:
light_source = np.array([0, 0, 0, 1])
x0 = np.array([
    10, 10, 10, # x,y,z
    1,0,0,0 # TODO
])

sensors = np.array([
    [ 1, 0, 0], # +x
    [-1, 0, 0], # -x
    [0,  1, 0], # +y
    [0, -1, 0], # -y
    [0, 0,  1], # +z
    [0, 0, -1], # -z
])

In [ ]:
dt, w_std, obs_std = 1e-1, 1e-3, 1e-2

## Simulation

In [ ]:
obs = Observation(sensors, light_source, obs_std, h)